### 뉴럴 스타일 트랜스퍼
* 스타일 참조 이미지, 베이스 이미지, 사전 훈련 모델 VGG19의 층 활성화를 동시에 계산하는 네트워크 설정
* 세 이미지에서 계산한 층 활성화를 사용하여 앞서 설명한 손실 함수를 정의
* 손실을 최소화하여 스타일 트랜스퍼 구현
* 손실 함수를 최소화할 경사 하강 과정 설정

In [2]:
from PIL import Image

# 이미지 파일 경로 설정
base_image_path = "test.jpg" 
style_reference_image_path = "rain.jpg" 

# 이미지 열기
ref_image = Image.open(style_reference_image_path)
base_image = Image.open(base_image_path)

# 이미지 크기 확인
width1, height1 = base_image.size
width2, height2 = ref_image.size
print(f"베이스 이미지 크기: {width1} x {height1}")
print(f"레퍼런스 이미지 크기: {width2} x {height2}")

베이스 이미지 크기: 1920 x 1080
레퍼런스 이미지 크기: 850 x 566


In [3]:
import os
print(os.getcwd())

c:\pycode\deep-learning-with-python-2nd-main\deep-learning-with-python-2nd-main\3rd_project


In [5]:
# 스타일 이미지와 베이스 이미지는 크기가 비슷하다
# 같은 높이가 되도록 크기 통일
from tensorflow import keras

# 레퍼런스 이미지 크기를 베이스 이미지 크기에 맞게 조정
resized_ref_image = ref_image.resize((512, 512))

# 리사이즈된 레퍼런스 이미지 확인
resized_ref_width, resized_ref_height = resized_ref_image.size
print(f"리사이즈된 레퍼런스 이미지 크기: {resized_ref_width} x {resized_ref_height}")

# 리사이즈된 이미지 저장 (원하는 경우)
resized_ref_image.save("./rain_ref.jpg")

리사이즈된 레퍼런스 이미지 크기: 512 x 512


In [6]:
# 이미지 로드, 전처리 사후 처리를 위한 유틸리티 함수
import numpy as np

# 이미지를 로드하고 크기를 바꾸어 적절한 배열로 변환하는 유틸리티함수
def preprocess_image(image_path):
    img = keras.utils.load_img(
        image_path, target_size=(512, 512))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img

# 넘파이를 배열 이미지로 변환
def deprocess_image(img):
    img = img.reshape((512, 512, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    return img

In [7]:
# 사전 훈련된 VGG19 모델로 특성 추출
model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# 모든 타깃 층의 활성화 값을 (하나의 딕셔너리로) 반환환
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

In [8]:
#콘텐츠 손실 
def content_loss(base_img, combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

In [12]:
# 스타일 손실
# 입력 행렬의 그람 행렬을 계산 : 원복 특성 행렬의 상관관계 기록
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style_img, combination_img):
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = 512 * 512
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [13]:
# 총 변위 손실 : 생성된 이미지의 픽셀을 사용하여 계산
# 픽셀의 격자 무늬가 과도하게 나타나는 것을 막는다.
def total_variation_loss(x):
    a = tf.square(
        x[:, : 512 - 1, : 512 - 1, :] - x[:, 1:, : 512 - 1, :]
    )
    b = tf.square(
        x[:, : 512 - 1, : 512 - 1, :] - x[:, : 512 - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

In [14]:
# 최소화할 최종 손실 정의
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layer_name = "block5_conv2"
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8 # 조정하여 얼마만큼 섞을 지 정하기기

def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=()) # 손실 초기화
    layer_features = features[content_layer_name] # 콘텐츠 손실 더하기
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    for layer_name in style_layer_names: # 스타일 손실 더하기
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(
          style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image) # 총 변위 손실 더하기기
    return loss

In [15]:
# 경사하강법 단계 설정 - 학습률 스케줄 옵티마이저 기능 활용

import tensorflow as tf

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))

iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print(f"{i}번째 반복: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"rainyday_{i}.png"
        keras.utils.save_img(fname, img)

c:\Users\acorn\anaconda3\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(3, 512, 512, 3))
  warnings.warn(msg)


100번째 반복: loss=7177.38
200번째 반복: loss=5394.07
300번째 반복: loss=4680.64


AbortedError: Graph execution error:

Detected at node functional_1/block3_conv4_1/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\acorn\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\acorn\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\acorn\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "c:\Users\acorn\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\acorn\anaconda3\Lib\asyncio\windows_events.py", line 322, in run_forever

  File "c:\Users\acorn\anaconda3\Lib\asyncio\base_events.py", line 641, in run_forever

  File "c:\Users\acorn\anaconda3\Lib\asyncio\base_events.py", line 1986, in _run_once

  File "c:\Users\acorn\anaconda3\Lib\asyncio\events.py", line 88, in _run

  File "c:\Users\a